In [1]:
import os
import json
from dotenv import load_dotenv
from pathlib import Path

dotenv_path = Path("../../.env")
if dotenv_path.exists():
    print("Loaded .env file!")
    load_dotenv(str(dotenv_path))


data = json.load(open(Path(os.environ["PROJECT_HOME_DIR"]) / "data/train.json"))

In [94]:
actual_names = []

for d in data:
    actual_names.append([x for x,y in zip(d["tokens"], d["labels"]) if "NAME_STUDENT" in y])

actual_names

[['Nathalie', 'Sylla', 'Nathalie', 'Sylla', 'Nathalie', 'Sylla'],
 ['Diego', 'Estrada', 'Diego', 'Estrada'],
 ['Gilberto', 'Gamboa'],
 ['Sindy', 'Samaca'],
 ['Nadine', 'Born'],
 ['Eladio', 'Amaya'],
 ['Silvia', 'Villalobos'],
 ['Sakir', 'Ahmad'],
 ['Francisco', 'Ferreira'],
 ['Stefano', 'Lovato'],
 ['Al'],
 ['Pepa', 'Medrano'],
 ['Deiby'],
 ['Fareed', 'Ponce'],
 ['Claudia', 'Sarria'],
 ['Rajinder', 'Santos'],
 ['Maud', 'Dias', 'Maud', 'Dias', 'Maud', 'Dias', 'Maud', 'Dias'],
 [],
 ['Zia'],
 ['Davide', 'Carletti'],
 ['Karan', 'Patel'],
 ['Milton', 'Desai'],
 ['Luis', 'Ramadan'],
 ['Cesar', 'Rivera'],
 ['Dharmendra', 'Asiri', 'Dharmendra', 'Asiri'],
 ['Daniel'],
 ['Suhag', 'Shah'],
 ['Eina', 'Nazim'],
 ['Mauro', 'Pacheco'],
 ['Gabriel',
  'Bravo',
  'Hlengiwe',
  'Swetha',
  'Tino',
  'Swetha',
  'Tino',
  'Lopez',
  'Swetha',
  'Swetha',
  'Alex',
  'Swetha',
  'Alex',
  'Bravo',
  'Tino',
  'Lopez',
  'Hlengiwe'],
 ['Mohd', 'Asim'],
 ['Wilson', 'Syafinaz', 'Wilson', 'Syafinaz'],
 ['Mad

In [61]:
from transformers import AutoModelForTokenClassification, AutoTokenizer, Trainer, TrainingArguments, pipeline

model_path = "/drive2/kaggle/pii-dd/piidd/training/names/outputs/d3b_lr_3e-5/checkpoint-524"

model = AutoModelForTokenClassification.from_pretrained(model_path, device_map=0)
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [62]:
pl = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

In [95]:
from tqdm.auto import tqdm
from datasets import Dataset


from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset
import datasets

dataset = Dataset.from_dict({"text": [d["full_text"] for d in data]})

names = []

# for out in tqdm(pl(KeyDataset(dataset, "text"), batch_size=8)):
#     names.append(out)

for d in tqdm(dataset[:100]["text"]):
    names.append(pl(d))

  0%|          | 0/100 [00:00<?, ?it/s]

/home/nicholas/miniconda3/lib/python3.10/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [41]:
data[0]["full_text"]

"Design Thinking for innovation reflexion-Avril 2021-Nathalie Sylla\n\nChallenge & selection\n\nThe tool I use to help all stakeholders finding their way through the complexity of a project is the  mind map.\n\nWhat exactly is a mind map? According to the definition of Buzan T. and Buzan B. (1999, Dessine-moi  l'intelligence. Paris: Les Éditions d'Organisation.), the mind map (or heuristic diagram) is a graphic  representation technique that follows the natural functioning of the mind and allows the brain's  potential to be released. Cf Annex1\n\nThis tool has many advantages:\n\n•  It is accessible to all and does not require significant material investment and can be done  quickly\n\n•  It is scalable\n\n•  It allows categorization and linking of information\n\n•  It can be applied to any type of situation: notetaking, problem solving, analysis, creation of  new ideas\n\n•  It is suitable for all people and is easy to learn\n\n•  It is fun and encourages exchanges\n\n•  It makes visi

In [43]:
len([x for x in names if len(x) > 0])

191

In [135]:
import random
random.choice(list(zip(range(len(names)), names, actual_names)))

(14, [], ['Claudia', 'Sarria'])

In [136]:
data[14]["full_text"]

"Mind Mapping (Module 3)\n\nChallenge\n\nIn the third part we use a tool called mind map that in principal means that you  have to put all the relevant information about a topic and this would help to  understand better the topic that is in the organizer. Also in this type of organizer  we can found a lot of kind of mind maps for example, in the classification of  them, the most important are Library mind maps for information organizing,  Presentation mind maps for presenting ideas and projects and Tunnel timeline  mind maps for organizing or making a project plan.\n\nSelection\n\nIt is a powerful graphic technique you can use to translate what's in  your mind into a visual picture. Since mind mapping works like the brain does it  allows you to organize and understand information faster and better. Then, in my  opinion I think that this tool is a very good option to take some decisions because  with this tool you can see the most relevant information and the main point about  a topic a

In [138]:
import torch

txt = """Reflection-Storytelling

Challenge

My friend Teresa was very anxious and unhappy recently and it was because of her son Naser. Naser is  10 years old and turned to be a little rebellious now. Due to the school closure in covid-19, Naser  has classes online and he then formed a habit of spending much on computers playing games or  watching videos he likes. Teresa was at work and can not monitor her son during working hours,  and she worried about Naser’s health as he spent too much time on computers so he completed his  homework very late, so he slept very late and his eyesight is poor as well. Naser used to be a good  student and obedient son, and he agreed to lessen time on computers or cellphones, but he can  not always keep his words. That’s why Teresa worried. She is afraid he turned to a bad boy."""

with torch.inference_mode():
    out = model(**tokenizer(txt, return_tensors="pt").to(0))

In [139]:
t = tokenizer.tokenize(txt, add_special_tokens=True)
print(t)

['[CLS]', '▁Reflection', '-', 'Story', 'telling', '▁Challenge', '▁My', '▁friend', '▁Teresa', '▁was', '▁very', '▁anxious', '▁and', '▁unhappy', '▁recently', '▁and', '▁it', '▁was', '▁because', '▁of', '▁her', '▁son', '▁Nas', 'er', '.', '▁Nas', 'er', '▁is', '▁10', '▁years', '▁old', '▁and', '▁turned', '▁to', '▁be', '▁a', '▁little', '▁rebellious', '▁now', '.', '▁Due', '▁to', '▁the', '▁school', '▁closure', '▁in', '▁co', 'vid', '-', '19', ',', '▁Nas', 'er', '▁has', '▁classes', '▁online', '▁and', '▁he', '▁then', '▁formed', '▁a', '▁habit', '▁of', '▁spending', '▁much', '▁on', '▁computers', '▁playing', '▁games', '▁or', '▁watching', '▁videos', '▁he', '▁likes', '.', '▁Teresa', '▁was', '▁at', '▁work', '▁and', '▁can', '▁not', '▁monitor', '▁her', '▁son', '▁during', '▁working', '▁hours', ',', '▁and', '▁she', '▁worried', '▁about', '▁Nas', 'er', '’', 's', '▁health', '▁as', '▁he', '▁spent', '▁too', '▁much', '▁time', '▁on', '▁computers', '▁so', '▁he', '▁completed', '▁his', '▁homework', '▁very', '▁late', ',',

In [142]:
pl(txt)

/home/nicholas/miniconda3/lib/python3.10/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


[{'entity_group': 'NAME',
  'score': 0.9283984,
  'word': 'Naser',
  'start': 120,
  'end': 126},
 {'entity_group': 'NAME',
  'score': 0.9362397,
  'word': 'Naser',
  'start': 127,
  'end': 133},
 {'entity_group': 'NAME',
  'score': 0.93965244,
  'word': 'Naser',
  'start': 231,
  'end': 237},
 {'entity_group': 'NAME',
  'score': 0.9480499,
  'word': 'Naser',
  'start': 450,
  'end': 456},
 {'entity_group': 'NAME',
  'score': 0.93483853,
  'word': 'Naser',
  'start': 601,
  'end': 607}]

In [140]:
s = out.logits.softmax(-1).reshape(-1, 2)

list(zip(t, s))

[('[CLS]', tensor([0.9986, 0.0014], device='cuda:0')),
 ('▁Reflection', tensor([9.9918e-01, 8.1709e-04], device='cuda:0')),
 ('-', tensor([9.9948e-01, 5.2194e-04], device='cuda:0')),
 ('Story', tensor([9.9950e-01, 5.0328e-04], device='cuda:0')),
 ('telling', tensor([9.9966e-01, 3.4206e-04], device='cuda:0')),
 ('▁Challenge', tensor([9.9952e-01, 4.7598e-04], device='cuda:0')),
 ('▁My', tensor([9.9927e-01, 7.3367e-04], device='cuda:0')),
 ('▁friend', tensor([9.9977e-01, 2.3051e-04], device='cuda:0')),
 ('▁Teresa', tensor([0.6272, 0.3728], device='cuda:0')),
 ('▁was', tensor([9.9991e-01, 9.3162e-05], device='cuda:0')),
 ('▁very', tensor([9.9982e-01, 1.7549e-04], device='cuda:0')),
 ('▁anxious', tensor([9.9979e-01, 2.0574e-04], device='cuda:0')),
 ('▁and', tensor([9.9993e-01, 6.9925e-05], device='cuda:0')),
 ('▁unhappy', tensor([9.9987e-01, 1.3465e-04], device='cuda:0')),
 ('▁recently', tensor([9.9978e-01, 2.2044e-04], device='cuda:0')),
 ('▁and', tensor([9.9974e-01, 2.5785e-04], device='c

In [69]:
data[18]["full_text"]

"Design Thinking for Innovation\n\nof University of Virginia\n\nFinal assignment by Zia\n\nVisualization Tool of Week 1 selected.\n\nWhat do have pizza and 5G in common ?\n\nChallenge: I am working at a international telco in the cellular business. We provide connectivity for  our customers via cellular networks since decades. As the application and the requirements of  customers besides plain telephony and SMS has significantly changed to “mobile Broadband” with  Smartphones in the 4th generation of cellular networks (4G, LTE), the 5th generation (5G) sets the aim  of getting the Industry connected. The demands of the industry are very different to the mobile  broadband demands as latency plays the most significant role (e.g. in a production environment). To  convince the management of changing certain things in the 4G networks to cope better with the  demands of 5G (and the vertical customers, i.e. industry) an approach was needed to explain the  need for certain changes.\n\nSelectio

In [63]:
pl("My name is Charmaine. Alex is my brother. And Uararo, a local artist, captures the lively scene on canvas, her brushstrokes bold and vibrant.")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity_group': 'NAME',
  'score': 0.6830238,
  'word': 'Alex',
  'start': 21,
  'end': 26},
 {'entity_group': 'NAME',
  'score': 0.98529845,
  'word': 'Uararo',
  'start': 45,
  'end': 52}]